# San Francisco Housing Cost Analysis

In this assignment, you will perform fundamental analysis for the San Francisco housing market to allow potential real estate investors to choose rental investment properties.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')

import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read the Mapbox API key
load_dotenv("../../../resources/mapbox_keys.env")
map_token = os.getenv("mapbox")
type(map_token)

## Load Data

In [ ]:
# Read the census data into a Pandas DataFrame
sfo_data_filepath = Path("data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(sfo_data_filepath, index_col="year")
sfo_data

- - - 

## Housing Units Per Year

In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function.

**Hint:** Use the Pandas `groupby` function.

**Optional challenge:** Use the min, max, and std to scale the y limits of the chart.



In [ ]:
# Calculate the mean number of housing units per year (hint: use groupby)
sfo_data_grouped_year = sfo_data.groupby(['year'])
sfo_data_mean_units = sfo_data_grouped_year['housing_units'].mean()
sfo_data_mean_units

In [ ]:
# Save the dataframe as a csv file
sfo_data.to_csv("sfo_data.csv")
sfo_data_mean_units.to_csv("sfo_data_mean_units.csv")

In [ ]:
# Use the Pandas plot function to plot the average housing units per year.
ax1 = sfo_data_mean_units.plot.bar(title="SF Mean Number of Housing Units Per Year (manual y-axis)")
ax1.set_ylim(360000, 388000)

In [ ]:
# Optional Challenge: Use the min, max, and std to scale the y limits of the chart
ax2 = sfo_data_mean_units.plot.bar(title="SF Mean Number of Housing Units Per Year (auto y-axis)")
bottom_lim = int(sfo_data_mean_units.min() - sfo_data_mean_units.std())
top_lim = int(sfo_data_mean_units.max() + sfo_data_mean_units.std())
ax2.set_ylim(bottom_lim, top_lim)

- - - 

## Average Housing Costs in San Francisco Per Year

In this section, you will calculate the average monthly rent and the average price per square foot for each year. An investor may wish to better understand the sales price of the rental property over time. For example, a customer will want to know if they should expect an increase or decrease in the property value over time so they can determine how long to hold the rental property.  Plot the results as two line charts.

**Optional challenge:** Plot each line chart in a different color.

In [ ]:
# Calculate the average sale price per square foot and average gross rent
gross_rent = sfo_data_grouped_year['gross_rent'].mean()
sale_price_sqr_foot = sfo_data.groupby(['year'])['sale_price_sqr_foot'].mean()
aggregate_data = pd.concat([gross_rent, sale_price_sqr_foot], axis='columns', join='inner')
aggregate_data

In [ ]:
# Create two line charts, one to plot the average sale price per square foot and another for average montly rent
fig, axs = plt.subplots(ncols=2, figsize=(10, 4))

# Line chart for average sale price per square foot
ax3 = sale_price_sqr_foot.plot(color='purple', title='Average Sale Price by Year', ax=axs[0])
ax3.set_ylabel('Price per Sqft')
ax3.set_xlabel('Year')

# Line chart for average montly rent
ax4 = gross_rent.plot(color='red', title='Average Gross Rent by Year', ax=axs[1])
ax4.set_ylabel('Price per Sqft')
ax4.set_xlabel('Year')

- - - 

## Average Prices by Neighborhood

In this section, you will use hvplot to create two interactive visulizations of average prices with a dropdown selector for the neighborhood. The first visualization will be a line plot showing the trend of average price per square foot over time for each neighborhood.  The second will be a line plot showing the trend of average montly rent over time for each neighborhood.

**Hint:** It will be easier to create a new DataFrame from grouping the data and calculating the mean prices for each year and neighborhood

In [ ]:
# Group by year and neighborhood and then create a new dataframe of the mean values
sfo_data_year_hood = sfo_data.groupby(['year','neighborhood']).mean()
sfo_data_year_hood

In [ ]:
# Use hvplot to create an interactive line chart of the average price per sq ft.
# The plot should have a dropdown selector for the neighborhood
sfo_data_year_hood_plot = sfo_data_year_hood.hvplot.line(x='year', y='sale_price_sqr_foot', groupby='neighborhood', ylabel='Avg. Sale Price per Square Foot')
sfo_data_year_hood_plot

In [ ]:
# Use hvplot to create an interactive line chart of the average monthly rent.
# The plot should have a dropdown selector for the neighborhood
sfo_data_year_hood_plot = sfo_data_year_hood.hvplot.line(x='year', y='gross_rent', groupby='neighborhood', ylabel='Avg. Gross Rent per Square Foot')
sfo_data_year_hood_plot

## The Top 10 Most Expensive Neighborhoods

In this section, you will need to calculate the mean sale price per square foot for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [ ]:
# Getting the data from the top 10 expensive neighborhoods to own (by sale price)
sfo_data_max10 = sfo_data.groupby(['neighborhood']).mean().nlargest(10,'sale_price_sqr_foot')
sfo_data_max10

In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
sfo_data_max10_plot = sfo_data_max10['sale_price_sqr_foot'].plot.bar(rot=90, title='Top 10 Expensive Neightborhoods in SFO', ylabel='Avg. SAle Price Per Square Foot')

- - - 

## Comparing cost to purchase versus rental income

In this section, you will use `hvplot` to create an interactive visualization with a dropdown selector for the neighborhood. This visualization will feature a side-by-side comparison of average price per square foot versus average montly rent by year.  

**Hint:** Use the `hvplot` parameter, `groupby`, to create a dropdown selector for the neighborhood.

In [ ]:
# Fetch the previously generated DataFrame that was grouped by year and neighborhood
sfo_data_year_hood

In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
sfo_data_top10_year_hood_plot = sfo_data_year_hood.hvplot.bar(x='year', y=['sale_price_sqr_foot','gross_rent'], groupby='neighborhood', ylabel='Num. Housing Units', rot=90, color=['red','blue'], height=500)
sfo_data_top10_year_hood_plot

- - - 

## Neighborhood Map

In this section, you will read in neighborhoods location data and build an interactive map with the average house value per neighborhood. Use a `scatter_mapbox` from Plotly express to create the visualization. Remember, you will need your Mapbox API key for this.

### Load Location Data

In [ ]:
# Load neighborhoods coordinates data
loc_data_filepath = Path("data/neighborhoods_coordinates.csv")
loc_data = pd.read_csv(loc_data_filepath).set_index('Neighborhood')
loc_data.head()

### Data Preparation

You will need to join the location data with the mean values per neighborhood.

1. Calculate the mean values for each neighborhood.

2. Join the average values with the neighborhood locations.

In [ ]:
# Calculate the mean values for each neighborhood
mean_data = sfo_data.groupby(['neighborhood']).mean()
mean_data.head()

In [ ]:
# Join the average values with the neighborhood locations
mean_data_loc = pd.concat([mean_data, loc_data], axis='columns', join='inner')
mean_data_loc.head()

### Mapbox Visualization

Plot the average values per neighborhood using a Plotly express `scatter_mapbox` visualization.

In [ ]:
# Set the mapbox access token
px.set_mapbox_access_token(map_token)

# Create a scatter mapbox to analyze neighborhood info
mapbox_plot = px.scatter_mapbox(
    mean_data_loc,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    zoom = 10)


mapbox_plot.show()

- - -

## Cost Analysis - Optional Challenge

In this section, you will use Plotly express to create visualizations that investors can use to interactively filter and explore various factors related to the house value of the San Francisco's neighborhoods. 

### Create a DataFrame showing the most expensive neighborhoods in San Francisco by year

In [ ]:
# Fetch the data from all expensive neighborhoods per year.
df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(sfo_data_max10.reset_index()["neighborhood"])]
df_expensive_neighborhoods_per_year.head()

### Create a parallel coordinates plot and parallel categories plot of most expensive neighborhoods in San Francisco per year


In [ ]:
# Parallel Categories Plot
plot_1 = px.parallel_coordinates(df_expensive_neighborhoods_per_year.reset_index(), 
                                 color='year', 
                                 title = 'San Francisco Housing Market',
                                 labels = {
                                     'year' : 'Year',
                                     'neighborhood' : 'Neighborhood',
                                     'sale_price_sqr_foot' : '$ per Sq. Foot',
                                     'housing_units' : 'Housing Units',
                                     'gross_rent' : 'Gross Rent'
                                 })
plot_1

In [ ]:
# Parallel Coordinates Plot
plot_2 = px.parallel_categories(df_expensive_neighborhoods_per_year.reset_index(), 
                                 color='year', 
                                 title = 'San Francisco Most Expensive Neighborhoods',
                                 labels = {
                                     'year' : 'Year',
                                     'neighborhood' : 'Neighborhood',
                                     'sale_price_sqr_foot' : '$ per Sq. Foot',
                                     'housing_units' : 'Housing Units',
                                     'gross_rent' : 'Gross Rent'
                                 })
plot_2

### Create a sunburst chart to conduct a costs analysis of most expensive neighborhoods in San Francisco per year

In [ ]:
# Sunburst Plot
plot_3 = px.sunburst(
    df_expensive_neighborhoods_per_year.reset_index(),
    path=['year','neighborhood'], 
    values='sale_price_sqr_foot')

plot_3